<a href="https://colab.research.google.com/github/seraffin/FailOmen/blob/master/Pola/Workspace_randomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

# Helper libraries
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls
!cd '/content/gdrive/My Drive/FailOmen'

gdrive	sample_data


In [0]:
import pandas as pd

#Importing dataset
dataset = pd.read_csv('/content/gdrive/My Drive/FailOmen/dataToML_full7.csv', index_col=False)

In [0]:
x = dataset.iloc[:, 0:23739].values
y = dataset.iloc[:, 23739:24281].values

In [0]:
failCount = 0
passCount = 0
failBuildCount = 0

for a in y:
  if a[np.argmin(a)] == 0 : failBuildCount += 1
  for b in a:
    if b == 0 : failCount += 1
    if b == 1 : passCount += 1
print (failBuildCount)
print (failCount)
print (passCount)
print (failCount / passCount * 100, '%', sep='')

1884
3125
5558337
0.056221851967593904%


## **UTILITIES:**


---

In [0]:
def count_distribution(prediction):
  #Distributions of argmins through all the predictions
  i = 1
  tab = [0] * 542
  for a in prediction:
    j = 0
    for b in a:
      if b < 1.0 : j = j + 1

    #print (i, '. ', j, np.argmin(a))
    tab[np.argmin(a)] += 1
    i = i + 1

  i = 0
  number_of_tests =  542
  distributed_array = [0 for iterator in range(number_of_tests)]
  for a in tab:
    if a > 0 : 
      pass
#       print ('position', i, '\targmin count', a)
      distributed_array[i] = a
    i += 1
#   print (i)
  create_plot(range(len(distributed_array)), distributed_array)  
  return distributed_array


In [0]:
def create_plot(x_data, y_data=0):
  import matplotlib.pyplot as plt
  if y_data == 0:  
    unzip = list(zip(*x_data))
    x_data, y_data = unzip[0],unzip[1]
    print (x_data)
    print ("\n", y_data)
  
  plt.bar(x_data, y_data, align='center', alpha=1, width = 3)
  plt.xlabel("test no")
  plt.ylabel("how many times test was predicted as probably failed")
  plt.show()

uwaga, w funkcji ponizej searchForFailed(), w wyborze dodania spassowanych testow do trainig/testing set-a, mino ze podajemy dokladny procent z spassowanych danych jaki chcemy wykorzysatc, procet ten moze sie roznic w rzeczywistosci. Spowodowane jest to warunkiem modulo, np 20% bedzie 20%, 90% bedzie 100% 

 

In [0]:
def searchForFailed(yList, percent_of_passed_data=20, withpassed_data=False): 
  changedRowList = []
  period_counter_for_passedData = 0
  for row in range (len(yList)):
    if 0 in yList[row]:
      changedRowList.append(row)
      continue
    elif withpassed_data == True: 
      modulo_arg = int(1/(percent_of_passed_data/100))
      if not (period_counter_for_passedData%modulo_arg):
        changedRowList.append(row)
    period_counter_for_passedData += 1
      
  return changedRowList

def returnFailedData(xList, yList, changedRowList):
  xFailed = []
  yFailed = []
  
  for row in changedRowList:
    xFailed.append(xList[row])
    yFailed.append(yList[row])
  xFailed = np.array(xFailed)
  yFailed = np.array(yFailed)
  return xFailed, yFailed

In [0]:
def failsCount():
  fala = 0
  for i, a in enumerate(yTest):
    j = 0
    for k, b in enumerate(a):
      if b < 1.0 : j += 1

    if j > 0 : 
      print (i, '. ', j)
    i = i + 1
    fala += j
  print (fala)

In [0]:
# Evaluate by probability - if the percentage parameter is set to 0.5 than 50% 
# of predictions will be taken into cosideration. When the strongest prediction 
# in given build is 0.9 then predictions from 0.9 to 0.95 will be 
# taken into consideration, if 0.8 than from 0.8 to 0.9

def evaluation2(percentage, predictions, refYsupervisor, verbose=False):
  
  lenght = len(refYsupervisor)
  failPositions = [[] for y in range(lenght)]
  failPredictions = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]
  predictionHits = [0] * lenght
  validPredictionsCount = [0] * lenght

  for i, a in enumerate(refYsupervisor):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);
  
  for i, a in enumerate(predictions):
    probabilityLimit = np.amin(a) + ((1 - np.amin(a)) * percentage)

    for j, b in enumerate(a):
      if b <= probabilityLimit: 
        predictionPositions[i].append(j)
        validPredictionsCount[i] += 1
        
  for i, a in enumerate(failPositions):
    count = 0
    for b in a:
      for c in predictionPositions[i]:
        if c == b : count += 1

    if len(a) != 0:
      predictionHits[i] = count

  hitsCount = 0
  failsCount = 0
  
  for i in range(lenght):
   # print(predictionHits[i])
    hitsCount += predictionHits[i]
    failsCount += len(failPositions[i])  
  print ("hits {}, liczba faktyczny faili {} ".format(hitsCount,failsCount ))

  print('Percentage of fails predicted', hitsCount / failsCount * 100, '%',
        '//Average of valid predictions', sum(validPredictionsCount)/len(validPredictionsCount))
  
  if verbose == True:
    for i in range(lenght):
      print(i ,len(failPositions[i]), len(predictionPositions[i]), predictionHits[i], sep='\t')
  performance = hitsCount/failsCount * 100
  return  performance

In [0]:
def evaluation_new(additionalPredictions, predictions, refYsupervisor, verbose = False):
  
  lenght = len(refYsupervisor)

  failPositions = [[] for y in range(lenght)]

  for i, a in enumerate(refYsupervisor):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);

  predictionsTemp = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]


  for i, a in enumerate(predictionsTemp):

    if len(failPositions[i]) != 0:
      for j in range(1 + additionalPredictions):
        argmin = np.argmin(a)
        predictionPositions[i].append(argmin)
        predictionsTemp[i][argmin] = 1

  predictionHits = [[] for y in range(lenght)]

  for i, a in enumerate(failPositions):
    count = 0
    for j, b in enumerate(a):


      for c in predictionPositions[i]:
        if c == b : count += 1

    if len(failPositions) != 0:
      predictionHits[i] = count


  failsCount = 0
  hitsCount = 0
  for i, a in enumerate(refYsupervisor):
    j = 0
    for k, b in enumerate(a):
      if b < 1.0 : j += 1

  #  if j > 0 :
  #    print (i, '.', j, predictionHits[i])
    failsCount += j
    hitsCount += predictionHits[i]
    
  print ("hits {}, liczba faktyczny faili {} ".format(hitsCount,failsCount ))
  print('Percentage of fails predicted', hitsCount / failsCount * 100, '%')
  
  if verbose == True:
    for i in range(lenght):
      print(i ,len(failPositions[i]), len(predictionPositions[i]), predictionHits[i], sep='\t')
  performance = hitsCount/failsCount * 100
  return  performance

In [0]:
# Evaluation function

def evaluation_old(predictions,additionalPredictions, refYsupervisor):
  
  lenght = len(refYsupervisor)
  failPositions = [[] for y in range(lenght)]

  for i, a in enumerate(refYsupervisor):
    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);#spisuje w listach(per commit) pozycje, gdzie sfailowal test [3,78,90, itp]

  predictionsTemp = predictions.copy()#tablica list, w ktotej sa przedstawione prawd wystapienia danej labeliki
  predictionPositions = [[] for y in range(lenght)]

  for i, commit in enumerate(predictionsTemp):
    if len(failPositions[i]) != 0:#jezeli w danym commicie byl wh jakis blad
      for j in range(len(failPositions[i]) + additionalPredictions):#tyle razy co sfailowanych testow + dodatkowe, przez nas zdefiniowane
        argmin = np.argmin(commit)# argmin = jest indeksem najmniejsza wartoscia dla predykcji sieci , czyt. indexem najbardziej prawdopodobnej
        predictionPositions[i].append(argmin)#umiesc index tej  najbardziej prawdopodobna wartosc, w tab dla danego komitu 
        predictionsTemp[i][argmin] = 1#nie berz pod uwage juz tej wartosci

  predictionHits = [[] for y in range(lenght)]

  for i, a in enumerate(failPositions):
    count = 0
    for j, b in enumerate(a):
      for c in predictionPositions[i]:
        if c == b : count += 1
    if len(failPositions) != 0:
      predictionHits[i].insert(0,count)


  failsCount = 0
  hitsCount = 0
  for i, a in enumerate(refYsupervisor):
    j = 0

    for k, b in enumerate(a):
      if b < 1.0 : j += 1

    failsCount += j
    hitsCount += predictionHits[i][0]

#   print('Percentage of fails predicted', hitsCount / failsCount * 100, '%')
  performance = hitsCount/failsCount * 100
  return  performance



---



---



---



## **PREPARE DATA**

In [0]:
#xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2)

**(optional) create only 'failed data'**

In [0]:
import numpy
xReduced, yReduced = returnFailedData(x, y, searchForFailed(y,percent_of_passed_data=1,withpassed_data=False))
# xReduced, yReduced = returnFailedData(x, y, searchForFailed(y))
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(xReduced, yReduced, test_size = 0.2, random_state=77)

In [0]:
def count_fail_to_pass_ratio(refy):
  failCount = 0
  passCount = 0
  failBuildCount = 0

  for a in y:
    if a[np.argmin(a)] == 0 : failBuildCount += 1
  print ("To training were used data : ")
  print ("failed builds: ", failBuildCount)
  passed_builds = len(refy) - failBuildCount
  print('passed builds: ', passed_builds)
  try:
    print ("failed to passed  ratio: ", failBuildCount / passed_builds * 100, '%', sep='')
  except:
    print ("no passed data in dataset")
  
  
count_fail_to_pass_ratio(yReduced)
  

  
  

To training were used data : 
failed builds:  1884
passed builds:  0
no passed data in dataset




---



---



---



## PREPARE MODEL

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import time
#12140
from sklearn.ensemble import RandomForestRegressor


Using TensorFlow backend.


In [0]:

# Instantiate model with 1000 decision trees
# start_time = time.time()

# number_of_trees = 10
# model = RandomForestRegressor(n_estimators = number_of_trees, random_state = 42)
#   # Train the model on training data
# model.fit(xTrain, yTrain)
# print("--- %s seconds ---" % (time.time() - start_time))

# predictions = model.predict(xTest)

In [0]:
#print(predictions)
# print (evaluation_new(15,predictions,yTest))
# print (evaluation_old(predictions,15,yTest))
# print (evaluation2(15,predictions,yTest))



---



---



---



## **TEST MODEL**

In [0]:
def random_forest(number_of_trees):
  from sklearn.ensemble import RandomForestRegressor
  # Instantiate model with 1000 decision trees
  model = RandomForestRegressor(n_estimators = number_of_trees, random_state = 42)
  start_time = time.time()
  # Train the model on training data
  model.fit(xTrain, yTrain)
  print("--- %s seconds ---" % (time.time() - start_time))
  predictions = model.predict(xTest)
  performance = evaluation_new(15,predictions,yTest)
  return performance
  

## OPTIMIZER

In [0]:
def optimizer(func, steps, minMax1, minMax2=False, split = 4, reduceSplit = False, searchMaximum = True):
  """
  -------------------------------
  Function can take 1 or 2 parameters to optimization.
  Default: 2 parameters (you need to set values for minMax1 and minMax2)
           For 1 parameter optimization set argument: "minMax2 = False"

  -------------------------------
  func        -->  enter your function there
                   IMPORTANT:
                   * if function has 1 parameter:  minMax1 = [min, max],   minMax2 = False
                   * if function has 2 parameters: minMax1 = [min1, max1], minMax2 = [min2, max2]
                   
  steps       -->  number of iterations of recurency
  
  minMax1/2   -->  list of min and max value of parameters to begin with (ex. [min, max])
  
  split        -->  number of parts in which function will split values of parameters
                   higher value -> better accuracy & longer calculation time
                   (Values range 2 -- 10)
                   
  reduceSplit -->  If True: with every repetition of recurency will reduce split by one
  
  -------------------------------
  """
  import matplotlib.pyplot as plt
  import math
  
  
  plotData = []
  iterationCounter = 0
  iterationsLeft = steps
  recurencyCounter = 0
  bestResult = None
  print(iterationsLeft)
  
  #--------------------

  def returnStepList(minValue, maxValue, nrSteps):
      step = int(math.fabs(maxValue - minValue)/nrSteps)
      actualMin = min([minValue, maxValue])
      actualMax = max([minValue, maxValue])
      return [actualMin + step*nr for nr in range(nrSteps + 1)]

  def resetIterators():
    global plotData
    global iterationCounter
    global recurencyCounter
    plotData = []
    iterationCounter = 0
    recurencyCounter = 0


  def create_plot_optimizer(x_data, y_data=0, xLabel = 'X', yLabel = 'Y'):
      import matplotlib.pyplot as plt
      if y_data == 0:  
          unzip = list(zip(*x_data))
          x_data, y_data = unzip[0],unzip[1]
      plt.plot(x_data, y_data)
      plt.xlabel(xLabel)
      plt.ylabel(yLabel)
      plt.show()


  def optimize(func, steps, minMax1, minMax2 = False, split = 4, reduceSplit = False, 
               searchMaximum = True, iLeft = None):
      
      global resetIterators
      global iterationCounter
      global iterationsLeft
      global recurencyCounter
      global bestResult
      
      localICounter = 0
      iL = iLeft
      sMax = searchMaximum
      singleParam = False
      bestPair = [None, None]
      resultList = []
      parametersList=[]

      if reduceSplit is False:
        newSplit = split
      else:
        if split > 2:
          newSplit = split - 1
        else:
          newSplit = 2
      if split > 10:
        raise ValueError("Too big value of 'split' parameter! This would highly increase number of iterations")
      if minMax2 is False:
        singleParam = True

      recurencyCounter += 1

      if isinstance(minMax1, list) and (isinstance(minMax2, list) or singleParam) :

          stepList1 = returnStepList(minMax1[0], minMax1[1], split)
          if singleParam:
            stepList2 = [1]
          else:
            stepList2 = returnStepList(minMax2[0], minMax2[1], split)
          
          nrIterations = len(stepList1)*len(stepList2)
          i1 = 0
          i2 = 0



          for el1 in stepList1:
              i2 = 0
              for el2 in stepList2:
                  iterationCounter += 1
                  localICounter += 1
                 

                  #----function----
                  if singleParam:
                    evaluation = func(el1)
                  else:
                    evaluation = func(el1, el2)
                  #----function----
                  print("---Iteration: {0}/{1} ----Recurency: {2}/{3} ---Evaluation:{4} ---Parameter: {5}".format(localICounter, \
                                                                                                                   nrIterations,\
                                                                                                                   recurencyCounter,\
                                                                                                                   iL,\
                                                                                                                   evaluation,\
                                                                                                                   el1))\

                  if len(resultList) > 0:
                      #----condition----
                      if sMax:
                        if evaluation >= max(resultList):
                            bestPair[0] = i1
                            bestPair[1] = i2
                      else:
                        if evaluation <= min(resultList):
                            bestPair[0] = i1
                            bestPair[1] = i2

                      #----condition----

                  else:
                    if singleParam:
                      bestPair[0] = i1
                    else:
                      bestPair[0] = i1
                      bestPair[1] = i2

                  plotData.append([evaluation, iterationCounter])

                  resultList.append(evaluation)
                  if singleParam:
                    parametersList.append([el1])
                  else:
                    parametersList.append([el1, el2])

                  i2 += 1
              i1 += 1

          #------------------------------------------------


          if bestPair[0] == stepList1.index(stepList1[0]):
              minMax1[0] = int(stepList1[bestPair[0]]/2)
              minMax1[1] = stepList1[bestPair[0] + 1]
  #                 print("best values of PARAM_1 are close to MIN value")

          elif bestPair[0] == stepList1.index(stepList1[-1]):
              minMax1[0] = stepList1[bestPair[0] - 1]
              minMax1[1] = stepList1[bestPair[0]]*2
  #                 print("best values of PARAM_1 are close to MAX value")


          else:
              minMax1[0] = stepList1[bestPair[0] - 1]
              minMax1[1] = stepList1[bestPair[0] + 1]

          #------------------------------------------------
          if not singleParam:
            if bestPair[1] == stepList2.index(stepList2[0]):
                minMax2[0] = int(stepList1[bestPair[1]]/2)
                minMax2[1] = stepList1[bestPair[1] + 1]
  #                 print("best values of PARAM_2 are close to MIN value")

            elif bestPair[1] == stepList2.index(stepList2[-1]):
                minMax2[0] = stepList1[bestPair[1] - 1]
                minMax2[1] = stepList1[bestPair[1]]*2
  #                 print("best values of PARAM_2 are close to MAX value")


            else:
                minMax2[0] = stepList2[bestPair[1] - 1]
                minMax2[1] = stepList2[bestPair[1] + 1]

          #------------------------------------------------
          if sMax:
            result = max(resultList)
            param1 = parametersList[resultList.index(max(resultList))][0]
            if not singleParam:
              param2 = parametersList[resultList.index(max(resultList))][1]
          else:
            result = min(resultList)
            param1 = parametersList[resultList.index(min(resultList))][0]
            if not singleParam:
              param2 = parametersList[resultList.index(min(resultList))][1]
                    
          if steps > 0:
            if singleParam:
              print("---------------------------------------------------------\n")
              optimize(func,
                       steps - 1,
                       [minMax1[0],minMax1[1]],
                       minMax2 = False,
                       split = newSplit,
                       reduceSplit=reduceSplit,
                       searchMaximum = sMax,
                       iLeft = iL )
            else:
              print("---------------------------------------------------------\n")
              optimize(func,
                       steps - 1,
                       [minMax1[0], minMax1[1]],
                       minMax2=[minMax2[0], minMax2[1]],
                       split = newSplit,
                       reduceSplit=reduceSplit,
                       searchMaximum = sMax,
                       iLeft = iL)
            
          else:
            if singleParam:
              print('\n------------------ACHIEVED RESULTS------------------\n')
              print("{:<20}{}\n{:<20}{}\n".format('RESULT: ',
                                                  result,
                                                  'PARAMETER 1: ',
                                                  param1))
              return [result, param1]
            else:
              print('\n------------------ACHIEVED RESULTS------------------\n')
              print("{:<20}{}\n{:<20}{}\n{:<20}{}\n".format('RESULT: ',
                                                            result,
                                                            'PARAMETER 1: ',
                                                            param1,
                                                            'PARAMETER 2: ',
                                                            param2))
              return [result, param1, param2]

      else:
          print("enter correct value!")
         
  resetIterators()
  result = optimize(func,
                    steps,
                    minMax1,
                    minMax2,
                    split,
                    reduceSplit,
                    searchMaximum = searchMaximum,
                    iLeft = steps+1)
  xData = [num[1] for num in plotData]
  yData = [num[0] for num in plotData]
  create_plot_optimizer(xData, yData, 'Iterations', 'Result')
  return result

# EXECUTION 

In [0]:
optimizer(random_forest, steps=1, minMax1=[333,500], minMax2=False, split=2, reduceSplit=True, searchMaximum=True)

1
